In [1]:
import os, sys
# os.environ['CUDA_VISIBLE_DEVICES'] = "2"
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib as plt
from PIL import Image
sys.path.append('../src')
from utils import *
from matplotlib.lines import Line2D
import math
import matplotlib.image as mpimg
import scipy.stats as stats
import scipy as sp
from gnet8_encoder import GNet8_Encoder
from torchmetrics import PearsonCorrCoef
os.chdir("..")

/export/raid1/home/kneel027/miniconda3/envs/SS/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
trial_data, y = load_nsd_mental_imagery(vector = "images", subject=1, mode="attention", stimtype="simple")
GNet = GNet8_Encoder(device="cuda:2",subject=1)
PeC1 = PearsonCorrCoef(num_outputs=1)


In [17]:
stim_brain_corr_epoch1 = []
stim_brain_corr_epoch2 = []
barrage_brain_corr_epoch1 = []
barrage_brain_corr_epoch2 = []
for trial in tqdm(trial_data):
    images = trial["barrage"]
    beta1 = trial["voxels"][0]
    beta2 = trial["voxels"][1]
    target_beta_prime = GNet.predict([trial["stimulus"]])[0]
    brain_corr_rep1 = PeC1(beta1, target_beta_prime)
    brain_corr_rep2 = PeC1(beta2, target_beta_prime)
    stim_brain_corr_epoch1.append(float(brain_corr_rep1))
    stim_brain_corr_epoch2.append(float(brain_corr_rep2))
    # if len(trial["barrage_stim"]) > 0:
    #     target_beta_prime_barrage = GNet.predict(trial["barrage_stim"])[0]
    #     brain_corr_rep3 = PeC1(beta1, target_beta_prime_barrage)
    #     brain_corr_rep4 = PeC1(beta2, target_beta_prime_barrage)
    #     print(brain_corr_rep3, brain_corr_rep4)
    beta_primes = GNet.predict(images)
    PeC = PearsonCorrCoef(num_outputs=beta_primes.shape[0])
    brain_corr_barrage1 = PeC(beta_primes.moveaxis(0,1), beta1.repeat(beta_primes.shape[0], 1).moveaxis(0, 1))
    brain_corr_barrage2 = PeC(beta_primes.moveaxis(0,1), beta2.repeat(beta_primes.shape[0], 1).moveaxis(0, 1))
    sorted_brain_corrs1, sorted_indices = torch.sort(torch.cat((brain_corr_barrage1, torch.tensor([brain_corr_rep1]))))
    sorted_brain_corrs2, sorted_indices = torch.sort(torch.cat((brain_corr_barrage2, torch.tensor([brain_corr_rep2]))))
    for bc in brain_corr_barrage1:
        barrage_brain_corr_epoch1.append(float(bc))
    for bc in brain_corr_barrage2:
        barrage_brain_corr_epoch2.append(float(bc))
    # Find the index of 'value' in the sorted tensor
    ranked_index1 = torch.nonzero(sorted_brain_corrs1 == brain_corr_rep1).item()
    ranked_index2 = torch.nonzero(sorted_brain_corrs2 == brain_corr_rep2).item()

    # print("Ranking of brain_corr_rep1:", ranked_index1)
    # print("Ranking of brain_corr_rep2:", ranked_index2)

print("Average correlation between stimulus and brain activity for epoch 1:", np.mean(stim_brain_corr_epoch1))
print("Average correlation between stimulus and brain activity for epoch 2:", np.mean(stim_brain_corr_epoch2))
print("Average correlation between barrage and brain activity for epoch 1:", np.mean(barrage_brain_corr_epoch1))
print("Average correlation between barrage and brain activity for epoch 2:", np.mean(barrage_brain_corr_epoch2))



100%|██████████| 48/48 [00:58<00:00,  1.23s/it]

Average correlation between stimulus and brain activity for epoch 1: 0.08494614891242236
Average correlation between stimulus and brain activity for epoch 2: -0.07908060585032217
Average correlation between barrage and brain activity for epoch 1: 0.06706854062994876
Average correlation between barrage and brain activity for epoch 2: -0.06099691218260493
